In [ ]:
import os
from datetime import datetime, timedelta, timezone

DOSYA = "gorevler.txt"
son_silinen_gorev = None

# Türkiye saatini almak için
def turkiye_saati():
    zaman_farki = timedelta(hours=3)
    tz = timezone(zaman_farki)
    return datetime.now(tz).strftime("%d-%m-%Y %H:%M")

def gorevleri_yukle():
    gorevler = []
    if os.path.exists(DOSYA):
        with open(DOSYA, "r", encoding="utf-8") as dosya:
            for satir in dosya:
                parcalar = satir.strip().split("|")
                if len(parcalar) == 3:
                    metin = parcalar[0].replace("Görev: ", "").strip()
                    oncelik = parcalar[1].replace("Öncelik: ", "").strip()
                    tarih = parcalar[2].replace("Tarih: ", "").strip()
                    gorevler.append({"metin": metin, "oncelik": oncelik, "tarih": tarih})
    return gorevler

def gorevleri_kaydet(gorevler):
    with open(DOSYA, "w", encoding="utf-8") as dosya:
        for g in gorevler:
            satir = f"Görev: {g['metin']} | Öncelik: {g['oncelik']} | Tarih: {g['tarih']}\n"
            dosya.write(satir)

def gorevleri_listele(gorevler):
    if len(gorevler) == 0:
        print("Henüz görev yok.")
    else:
        print(f"\n--- GÖREV LİSTESİ ({len(gorevler)} görev) ---")
        for i, g in enumerate(gorevler, start=1):
            print(f"{i}. Görev: {g['metin']} | Öncelik: {g['oncelik']} | Tarih: {g['tarih']}")
        print("---")

def gorev_ekle(gorevler):
    metin = input("Yeni görev: ").strip()
    if metin == "":
        print("Boş görev eklenemez!")
        return
    print("Öncelik seçiniz: 1) Düşük 2) Orta 3) Yüksek")
    secim = input("Seçim: ")
    if secim == "1":
        oncelik = "Düşük"
    elif secim == "2":
        oncelik = "Orta"
    elif secim == "3":
        oncelik = "Yüksek"
    else:
        oncelik = "Orta"
    tarih = turkiye_saati()
    gorevler.append({"metin": metin, "oncelik": oncelik, "tarih": tarih})
    print(f"'{metin}' görevi eklendi.")
    gorevleri_kaydet(gorevler)

def gorev_duzenle(gorevler):
    gorevleri_listele(gorevler)
    try:
        secim = int(input("Düzenlemek istediğiniz görev numarası: "))
        if 1 <= secim <= len(gorevler):
            yeni_metin = input(f"Yeni görev metni ({gorevler[secim-1]['metin']}): ").strip()
            if yeni_metin != "":
                gorevler[secim-1]["metin"] = yeni_metin
                print("Görev güncellendi.")
                gorevleri_kaydet(gorevler)
            else:
                print("Boş görev girilemez!")
        else:
            print("Geçersiz numara!")
    except ValueError:
        print("Lütfen sayı girin!")

def gorev_sil(gorevler):
    global son_silinen_gorev
    gorevleri_listele(gorevler)
    try:
        secim = int(input("Silmek istediğiniz görev numarası: "))
        if 1 <= secim <= len(gorevler):
            son_silinen_gorev = gorevler.pop(secim-1)
            print(f"'{son_silinen_gorev['metin']}' görevi silindi.")
            gorevleri_kaydet(gorevler)
        else:
            print("Geçersiz numara!")
    except ValueError:
        print("Lütfen sayı girin!")

def geri_al(gorevler):
    global son_silinen_gorev
    if son_silinen_gorev:
        gorevler.append(son_silinen_gorev)
        print(f"'{son_silinen_gorev['metin']}' geri alındı.")
        son_silinen_gorev = None
        gorevleri_kaydet(gorevler)
    else:
        print("Geri alınacak görev yok.")

def gorev_ara(gorevler):
    aranan = input("Aramak istediğiniz kelime: ").strip().lower()
    bulunan = [g for g in gorevler if aranan in g['metin'].lower()]
    if bulunan:
        print(f"\n--- Arama Sonuçları ---")
        for i, g in enumerate(bulunan, start=1):
            print(f"{i}. Görev: {g['metin']} | Öncelik: {g['oncelik']} | Tarih: {g['tarih']}")
    else:
        print("Görev bulunamadı.")

def gorev_sirala(gorevler):
    print("1) Alfabetik 2) Öncelik 3) Tarihe göre")
    secim = input("Seçiminiz: ")
    if secim == "1":
        gorevler.sort(key=lambda g: g['metin'].lower())
    elif secim == "2":
        oncelik_sirala = {"Yüksek":1, "Orta":2, "Düşük":3}
        gorevler.sort(key=lambda g: oncelik_sirala[g['oncelik']])
    elif secim == "3":
        gorevler.sort(key=lambda g: datetime.strptime(g['tarih'], "%d-%m-%Y %H:%M"))
    else:
        print("Geçersiz seçim!")
        return
    print("Görevler sıralandı.")
    gorevleri_kaydet(gorevler)

def menu():
    gorevler = gorevleri_yukle()
    while True:
        print("\n--- TO-DO LIST ---")
        print("1. Görevleri Listele")
        print("2. Yeni Görev Ekle")
        print("3. Görev Düzenle")
        print("4. Görev Sil")
        print("5. Son Silinen Görevi Geri Al")
        print("6. Görev Ara")
        print("7. Görevleri Sırala")
        print("8. Çıkış")
        secim = input("Seçiminiz: ")

        if secim == "1":
            gorevleri_listele(gorevler)
        elif secim == "2":
            gorev_ekle(gorevler)
        elif secim == "3":
            gorev_duzenle(gorevler)
        elif secim == "4":
            gorev_sil(gorevler)
        elif secim == "5":
            geri_al(gorevler)
        elif secim == "6":
            gorev_ara(gorevler)
        elif secim == "7":
            gorev_sirala(gorevler)
        elif secim == "8":
            print("Programdan çıkılıyor...")
            break
        else:
            print("Geçersiz seçim!")

menu()